In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [8]:
from functools import reduce
from tqdm import tqdm_notebook as tqdm
import itertools
import math
with open('tsp.txt', 'r') as f:
    data = f.read().splitlines()[1:]
    

cities = [(float(x.split(" ")[0]), float(x.split(" ")[1])) for x in data]
distances = [[math.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2) for c1 in cities] for c2 in cities]

### With history for backtracking for optimal path -- requires >=32gb ram

In [66]:
num_cities = len(cities) - 10 # remove - 10 for run on complete dataset, requires 32gb or more ram

#StateMap contains (visited cities state, current city) : (distance, previous city) representation
#Initialised from state 0 with distance to each city
StateMap = {(1 << n, n): (distances[0][n], 0) for n in range(1, num_cities)}

for number_visited_cities in tqdm(range(2, num_cities)): # each subset of visited cities
    for visited_cities_combinations in itertools.combinations(range(1, num_cities), number_visited_cities): #every combination of visited cities (e.g. 1001, 1010, 1001... for 4C2)
        visited_cities_state = reduce(lambda x, y: x | (1<<y), [0] + list(visited_cities_combinations)) #turn integer list into set bits for visited state representation (e.g. [0, 1, 4] -> b10011)

        for previous_city in visited_cities_combinations: # every previous visited state to get to this state
            previous_visited_cities_state = visited_cities_state & ~(1 << previous_city) # get subset with one state missing
            
            #StateMap (visited_cities, previous_city) = minimum distance to get from any previous state to current city
            StateMap[(visited_cities_state, previous_city)] = min([(StateMap[(previous_visited_cities_state, current_city)][0] + distances[current_city][previous_city], current_city) for current_city in visited_cities_combinations if (current_city !=0 and current_city != previous_city)])

min([(StateMap[(2**num_cities - 2, n)][0] + distances[n][0], n) for n in range(1, num_cities)]) # 2**num_cities - 2 = state representation for everything but the initial city

(19269.93599779425, 4)

### Just the optimal tour length

In [65]:
num_cities = len(cities)

#StateMap contains (visited cities state, current city) : (distance, previous city) representation
#Initialised from state 0 with distance to each city
StateMap = {(1 << n, n): distances[0][n] for n in range(1, num_cities)}

for number_visited_cities in tqdm(range(2, num_cities)): # each subset of visited cities
    newStateMap = {}
    for visited_cities_combinations in itertools.combinations(range(1, num_cities), number_visited_cities): #every combination of visited cities (e.g. 1001, 1010, 1001... for 4C2)
        visited_cities_state = reduce(lambda x, y: x | (1<<y), [0] + list(visited_cities_combinations)) #turn integer list into set bits for visited state representation (e.g. [0, 1, 4] -> b10011)

        for previous_city in visited_cities_combinations: # every previous visited state to get to this state
            previous_visited_cities_state = visited_cities_state & ~(1 << previous_city) # get subset with one state missing
            
            #StateMap (visited_cities, previous_city) = minimum distance to get from any previous state to current city
            newStateMap[(visited_cities_state, previous_city)] = min([StateMap[(previous_visited_cities_state, current_city)] + distances[current_city][previous_city] for current_city in visited_cities_combinations if (current_city !=0 and current_city != previous_city)])
    
    del StateMap
    StateMap = newStateMap

min([StateMap[(2**num_cities - 2, n)] + distances[n][0] for n in range(1, num_cities)]) # 2**num_cities - 2 = state representation for everything but the initial city

26442.73030895475